# Phase 2 Project:

**Author**: Jordana Tepper

**Pace**: Live NYC

**Instructor**: Joseph Mata

## Overview

## Business Problem

## Data Understanding

In [505]:
#Importing neccesary packages

# Basic imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Data visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Pre-Processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#Modeling
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [506]:
#Loading in data from kc_house_data file which includes information about King County House Sales

kc_house_df = pd.read_csv('data/kc_house_data.csv')
kc_house_df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.00000,3,1.00000,1180,5650,1.00000,NaN,NONE,...,7 Average,1180,0.0,1955,0.00000,98178,47.51120,-122.25700,1340,5650
1,6414100192,12/9/2014,538000.00000,3,2.25000,2570,7242,2.00000,NO,NONE,...,7 Average,2170,400.0,1951,1991.00000,98125,47.72100,-122.31900,1690,7639
2,5631500400,2/25/2015,180000.00000,2,1.00000,770,10000,1.00000,NO,NONE,...,6 Low Average,770,0.0,1933,nan,98028,47.73790,-122.23300,2720,8062
3,2487200875,12/9/2014,604000.00000,4,3.00000,1960,5000,1.00000,NO,NONE,...,7 Average,1050,910.0,1965,0.00000,98136,47.52080,-122.39300,1360,5000
4,1954400510,2/18/2015,510000.00000,3,2.00000,1680,8080,1.00000,NO,NONE,...,8 Good,1680,0.0,1987,0.00000,98074,47.61680,-122.04500,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.00000,3,2.50000,1530,1131,3.00000,NO,NONE,...,8 Good,1530,0.0,2009,0.00000,98103,47.69930,-122.34600,1530,1509
21593,6600060120,2/23/2015,400000.00000,4,2.50000,2310,5813,2.00000,NO,NONE,...,8 Good,2310,0.0,2014,0.00000,98146,47.51070,-122.36200,1830,7200
21594,1523300141,6/23/2014,402101.00000,2,0.75000,1020,1350,2.00000,NO,NONE,...,7 Average,1020,0.0,2009,0.00000,98144,47.59440,-122.29900,1020,2007
21595,291310100,1/16/2015,400000.00000,3,2.50000,1600,2388,2.00000,NaN,NONE,...,8 Good,1600,0.0,2004,0.00000,98027,47.53450,-122.06900,1410,1287


In [507]:
#Looking at the general structure and content of kc_house_df

kc_house_df.info()

#21 columns and 21597 rows with missing values in 'waterfront', 'view', and 'yr_renovated'

#Datatypes in this DataFrame are int, object, and float

#For the columns with missing values, I will look at what percentage of the column this missing-ness accounts for
#After doing so, I will decide how to deal with the missing data

#'date' is listed as type object which is likely due to the way that the dates are formatted, 
# but I will further analyze this column to confirm

#It is also interesting that 'sqft_basement' is type object, so I will further explore this column.

#'zipcode' is of type int, but I know that a zipcode represents a given area and is actually a categorical variable. 
#Therefore, after I train test split, I will use OneHotEncoder create dummy variables for this column.
#Doing so will allow me to use 'zipcode' in my model despite it being a categorical variable.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [508]:
#Looking at what percent of the data in 'waterfront' is missing
kc_house_df['waterfront'].isnull().sum()/len(kc_house_df['waterfront'])

#Dropping 11% of the data is a lot, so I will use the 'view' column in our Data Preparation to minimize the number
#of entries that I drop.

#The column 'view' refers to the quality of the view from the house.
#More specifically, if the house has a view of a body of water, 
#the 'view' column will have a value that indicates a quality other than 'NONE' ('NONE' signfies no view)

#Therefore, if a an entry has a value of 'NONE' in 'view',
#I can assume that the value for 'waterfront' should be 'NO'

0.11001527989998611

In [509]:
#Looking at what percent of the data in 'view' is missing
kc_house_df['view'].isnull().sum()/len(kc_house_df['view'])

#Dropping 0.3% of the data will not make an impact on the rest of the data, so I will go ahead and drop the missing
#values from the 'view' column during my Data Preparation

0.0029170718155299346

In [510]:
#Looking at what percent of the data in 'yr_renovated' is missing
kc_house_df['yr_renovated'].isnull().sum()/len(kc_house_df['yr_renovated'])

#Dropping 17.8% of the data is a lot, but there is no other column that can provide me with this missing information,
#and I don't want to assume that missing data means that a house was not renovated.

#(Using 'yr_built' to fill in the missing data in 'yr_renovated' would be based upon the assumption that a missing
#entry means that the house was no renovated)

#Therefore, in conclusion, I'd rather have less (but still a good amount) data that is of higher accuracy 
#than slightly more data that is potentially wrong, so I will drop the missing values from 'yr_renovated' during
#my Data Preparation

0.17789507802009538

In [511]:
#Exploring the 'date' column Datatype
print(kc_house_df['date'])
print('\n')
print("A single entry in column 'date' is of'", type(kc_house_df['date'][0]))

#This shows me that the entries in the 'date' column are strings, so during my Data Preparation, 
#I will clean up this column by splitting it into a month column and a year column - both as type int

#After this, I will also confirm that this data about houses sold in year 2014 - 2015

0        10/13/2014
1         12/9/2014
2         2/25/2015
3         12/9/2014
4         2/18/2015
            ...    
21592     5/21/2014
21593     2/23/2015
21594     6/23/2014
21595     1/16/2015
21596    10/15/2014
Name: date, Length: 21597, dtype: object


A single entry in column 'date' is of' <class 'str'>


In [512]:
#Looking at the different values in the 'sqft_basement' column
print('Value Counts')
print(kc_house_df['sqft_basement'].value_counts())
print('\n')

print('Value Counts in Percentages')
print(kc_house_df['sqft_basement'].value_counts(normalize = True))

#I see that there are 454 entries in this column with the value '?' 
#This indicates that there is a placeholder in the 'sqft_basement' column to signify missing values

#Futhermore, when looking at the value counts as percentages, I can see that the '?' placeholders take up only 2.1%
#of the data, so during my data cleaning, it seems safe to drop these rows.

#After I drop these rows, I will convert the rest of the data from this column to floats

Value Counts
0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
2300.0        1
2610.0        1
2500.0        1
3500.0        1
2250.0        1
Name: sqft_basement, Length: 304, dtype: int64


Value Counts in Percentages
0.0      0.59388
?        0.02102
600.0    0.01005
500.0    0.00968
700.0    0.00963
           ...  
2300.0   0.00005
2610.0   0.00005
2500.0   0.00005
3500.0   0.00005
2250.0   0.00005
Name: sqft_basement, Length: 304, dtype: float64


In [513]:
#Deactivating scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

#Looking at the descriptive statistics for the numerical columns 
#(not including 'sqft_basement' because I have not yet dropped the '?' placeholders)

kc_house_df.describe()

#I see that many columns have outliers:
# - 'price' has outliers for both the min ($78,000) and max ($7,700,000)

# - 'bedrooms' has a max outlier (33) which I will probably remove during my Data Preparation

# - 'bathrooms' has a max outlier (8)

# - 'sqft_living' has outliers for both the min (370) and max (13,540)

# - 'sqft_lot' has outliers for both the min (520) and max (1,651,359)

# - 'sqft_above' has outliers for both the min (370) and max (9,410)

# - Something interesting about 'yr_renovated' is that the max is 2015 
#   but the min and 25%, 50%, and 75% quartiles are 0.
#   This likely indicates that a value of 0 means that the house has not been renovated, 
#   but I will do further analysis of this.

# - 'sqft_living15' - which is the square footage of interior housing living space for the nearest 15 neighbors -
#   has outliers for both the min (399) and max (6,210)

# - 'sqft_lot15' - which is the square footage of the land lots of the nearest 15 neighbors -
#   has outliers for both the min (651) and max (871,200)

#After I clean the 'sqft_basement' column in my Data Preparation, I will repeat the .describe() step

#Furthemore, this information shows me that I will eventually need to use StandardScaler 
#because the values in different columns are not of the same magnitude

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,21597.00000,21597.00000,21597.00000,21597.00000,21597.00000,21597.00000,21597.00000,21597.00000,21597.00000,17755.00000,21597.00000,21597.00000,21597.00000,21597.00000,21597.00000
mean,4580474287.77099,540296.57351,3.37320,2.11583,2080.32185,15099.40876,1.49410,1788.59684,1970.99968,83.63678,98077.95185,47.56009,-122.21398,1986.62032,12758.28351
std,2876735715.74778,367368.14010,0.92630,0.76898,918.10613,41412.63688,0.53968,827.75976,29.37523,399.94641,53.51307,0.13855,0.14072,685.23047,27274.44195
min,1000102.00000,78000.00000,1.00000,0.50000,370.00000,520.00000,1.00000,370.00000,1900.00000,0.00000,98001.00000,47.15590,-122.51900,399.00000,651.00000
25%,2123049175.00000,322000.00000,3.00000,1.75000,1430.00000,5040.00000,1.00000,1190.00000,1951.00000,0.00000,98033.00000,47.47110,-122.32800,1490.00000,5100.00000
50%,3904930410.00000,450000.00000,3.00000,2.25000,1910.00000,7618.00000,1.50000,1560.00000,1975.00000,0.00000,98065.00000,47.57180,-122.23100,1840.00000,7620.00000
75%,7308900490.00000,645000.00000,4.00000,2.50000,2550.00000,10685.00000,2.00000,2210.00000,1997.00000,0.00000,98118.00000,47.67800,-122.12500,2360.00000,10083.00000
max,9900000190.00000,7700000.00000,33.00000,8.00000,13540.00000,1651359.00000,3.50000,9410.00000,2015.00000,2015.00000,98199.00000,47.77760,-121.31500,6210.00000,871200.00000


In [514]:
#Examing the 'yr_renovated' column to see if a value of 0 represents no renovation

kc_house_df[['yr_built','yr_renovated']].sort_values(by = 'yr_built').head(30)

#It seems as though a value of 0 signfies no renovations because I can see that there are both NaN values and 0 values
#This means that a value of NaN represents missing data and a value of 0 must mean the absence of renovations

#Note: I included 'yr_built' in this evalutation because I thought that maybe a value of zero 
#      was specifically for newer houses that did not need renovations,
#      but as seen from the data, older homes also have values of 0.

,yr_built,yr_renovated
14069,1900,0.00000
14783,1900,0.00000
10973,1900,0.00000
115,1900,nan
4693,1900,nan
15283,1900,0.00000
8849,1900,1996.00000
18061,1900,0.00000
5231,1900,0.00000
12357,1900,0.00000


## Data Preparation

### Dropping the missing values in 'view'

In [515]:
#Dropping the missing values from 'view'
kc_house_df.dropna(subset = ['view'], axis = 0, inplace = True)
kc_house_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21534 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21534 non-null  int64  
 1   date           21534 non-null  object 
 2   price          21534 non-null  float64
 3   bedrooms       21534 non-null  int64  
 4   bathrooms      21534 non-null  float64
 5   sqft_living    21534 non-null  int64  
 6   sqft_lot       21534 non-null  int64  
 7   floors         21534 non-null  float64
 8   waterfront     19164 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21534 non-null  object 
 11  grade          21534 non-null  object 
 12  sqft_above     21534 non-null  int64  
 13  sqft_basement  21534 non-null  object 
 14  yr_built       21534 non-null  int64  
 15  yr_renovated   17704 non-null  float64
 16  zipcode        21534 non-null  int64  
 17  lat            21534 non-null  float64
 18  long  

### Filling in the missing values from 'waterfront' using the data from 'view'

In [516]:
#When 'view' has a value of 'NONE', assign the corresponding value of 'waterfront' to 'NO' 
#(including those that already have a value of 'NO')

kc_house_df.loc[kc_house_df['view'] == 'NONE', 'waterfront'] = 'NO'

In [517]:
#Checking how many missing values there are in 'waterfront' after this imputation
kc_house_df['waterfront'].isna().sum()/len(kc_house_df['waterfront'])

#I will drop the rest of the missing values in 'waterfront' because they only account for 1.2% of the data

0.012073929599702795

In [518]:
#Dropping the remaining missing values in 'waterfront'
kc_house_df.dropna(subset = ['waterfront'], axis = 0, inplace = True)
kc_house_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21274 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21274 non-null  int64  
 1   date           21274 non-null  object 
 2   price          21274 non-null  float64
 3   bedrooms       21274 non-null  int64  
 4   bathrooms      21274 non-null  float64
 5   sqft_living    21274 non-null  int64  
 6   sqft_lot       21274 non-null  int64  
 7   floors         21274 non-null  float64
 8   waterfront     21274 non-null  object 
 9   view           21274 non-null  object 
 10  condition      21274 non-null  object 
 11  grade          21274 non-null  object 
 12  sqft_above     21274 non-null  int64  
 13  sqft_basement  21274 non-null  object 
 14  yr_built       21274 non-null  int64  
 15  yr_renovated   17493 non-null  float64
 16  zipcode        21274 non-null  int64  
 17  lat            21274 non-null  float64
 18  long  

### Dropping missing values from 'yr_renovated'

In [519]:
#Dropping the missing values from 'yr_renovated'
kc_house_df.dropna(subset = ['yr_renovated'], axis = 0, inplace = True)
kc_house_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17493 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             17493 non-null  int64  
 1   date           17493 non-null  object 
 2   price          17493 non-null  float64
 3   bedrooms       17493 non-null  int64  
 4   bathrooms      17493 non-null  float64
 5   sqft_living    17493 non-null  int64  
 6   sqft_lot       17493 non-null  int64  
 7   floors         17493 non-null  float64
 8   waterfront     17493 non-null  object 
 9   view           17493 non-null  object 
 10  condition      17493 non-null  object 
 11  grade          17493 non-null  object 
 12  sqft_above     17493 non-null  int64  
 13  sqft_basement  17493 non-null  object 
 14  yr_built       17493 non-null  int64  
 15  yr_renovated   17493 non-null  float64
 16  zipcode        17493 non-null  int64  
 17  lat            17493 non-null  float64
 18  long  

### Cleaning the 'date' column

In [520]:
#Making a new column 'month_sold' that includes only the month value from the corresponding 'date' entry

#Splitting the 'date' entries at the '/' to make each entry a list that consists of three values: month, day, year
kc_house_df['month_sold'] = kc_house_df['date'].str.split('/')

#Using .apply so that each entry in the new column 'month_sold' includes only the month in which the house was sold
kc_house_df['month_sold'] = kc_house_df['month_sold'].apply(lambda x : x[0])

In [521]:
#Making a new column 'year_sold' that includes only the year value from the corresponding 'date' entry
#This is a very similar process to making the 'month_sold' column

#Splitting the 'date' entries at the '/' to make each entry a list that consists of three values: month, day, year
kc_house_df['year_sold'] = kc_house_df['date'].str.split('/')

#Using .apply so that each entry in the new column 'year_sold' includes only the year in which the house was sold
kc_house_df['year_sold'] = kc_house_df['year_sold'].apply(lambda x : x[2])

In [522]:
#Confirming for my own understanding that the house sales in the dataset are from the years 2014-2015
print(kc_house_df['year_sold'].min())
print(kc_house_df['year_sold'].max())

2014
2015


In [523]:
#Because I seperated 'date' in month and year, I will drop the 'date' column
kc_house_df.drop(columns = ['date'], inplace = True)

### Cleaning the 'sqft_basement' column

In [524]:
#Reassigning the original DataFrame so that it only includes the entries where 'sqft_basement' does not have a '?'
kc_house_df = kc_house_df[kc_house_df['sqft_basement'] != '?']

In [525]:
#Converting the remaining values in 'sqft_basement' to floats using .apply()
kc_house_df['sqft_basement'] = kc_house_df['sqft_basement'].apply(lambda x : float(x))

In [526]:
#During my Data Understanding, I mentioned that I would repeat the .describe() step 
#after I cleaned the 'sqft_basement' column and converted the remaining values to floats.

#I already examined the outliers from the rest of the numerical columns, so here I will focus on 'sqft_basement'.

kc_house_df.describe()

#sqft_basement has an outlier as its max (4820)
#The fact that the min of 'sqft_basement' is 0 tells me that this value signfies that there is no basement.

#Therefore, I will make a new column called 'basement' with values 1 and 0 
#where 1 means there is a basement and 0 means there is not basement.

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000,17131.00000
mean,4590416961.06252,536565.66067,3.37634,2.11650,2077.14261,15172.34149,1.49533,1788.91997,288.22264,1971.19187,81.33296,98077.51427,47.55960,-122.21286,1983.31849,12773.73989
std,2875969100.33560,364046.53305,0.93122,0.76462,912.38306,41961.30101,0.53859,825.31818,440.37654,29.29859,394.66046,53.40234,0.13868,0.14070,681.14260,27453.25674
min,1000102.00000,80000.00000,1.00000,0.50000,370.00000,520.00000,1.00000,370.00000,0.00000,1900.00000,0.00000,98001.00000,47.15590,-122.51900,399.00000,659.00000
25%,2126059294.50000,320000.00000,3.00000,1.75000,1430.00000,5037.00000,1.00000,1200.00000,0.00000,1952.00000,0.00000,98033.00000,47.46920,-122.32700,1490.00000,5100.00000
50%,3905040040.00000,450000.00000,3.00000,2.25000,1910.00000,7607.00000,1.50000,1560.00000,0.00000,1975.00000,0.00000,98065.00000,47.57140,-122.22800,1840.00000,7620.00000
75%,7334401020.00000,640000.00000,4.00000,2.50000,2540.00000,10624.00000,2.00000,2210.00000,550.00000,1997.00000,0.00000,98117.00000,47.67790,-122.12400,2360.00000,10074.50000
max,9895000040.00000,7700000.00000,33.00000,8.00000,13540.00000,1651359.00000,3.50000,9410.00000,4820.00000,2015.00000,2015.00000,98199.00000,47.77760,-121.31500,6210.00000,871200.00000


In [527]:
#Making the new column 'basement' where 0 means no basement and 1 means yes basement

#When 'sqft_basement' is greater than 0, the value of the new column 'basement' is 1
kc_house_df.loc[kc_house_df['sqft_basement'] > 0, 'basement'] = 1

#When 'sqft_basement' equals 0, the value of the new column 'basement' is 0
kc_house_df.loc[kc_house_df['sqft_basement'] == 0, 'basement'] = 0

#Making the 1's and 0's in the 'basement' column integers instead of floats
kc_house_df['basement'] = kc_house_df['basement'].apply(lambda x: int(x))

### Removing the extreme outlier from 'bedrooms'

In [528]:
#As seen from .describe(), there is an extreme outlier of 33 bedrooms in the 'bedrooms' column

#While this is likely an imputation error, I want to confirm this by looking at the number of bathrooms in the house
kc_house_df.loc[kc_house_df['bedrooms'] == 33]

#I see that there are 1.75 bathrooms in this house meaning that the value of 33 in 'bedrooms' cannot be accurate

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month_sold,year_sold,basement
15856,2402100895,640000.00000,33,1.75000,1620,6000,1.00000,NO,NONE,Very Good,...,1947,0.00000,98103,47.68780,-122.33100,1330,4700,6,2014,1


In [529]:
#Dropping the row with 33 bedrooms because its value is incorrect
kc_house_df.drop(index = 15856, inplace = True)

## Data Analysis

Before I begin my modeling, I want to visualize some of the trends in the data.

### Visualizing the relationship between 'price' and 'zipcode'

In [530]:
#HELP
#price_x_zipcode = kc_house_df.groupby('zipcode')['price'].mean()
#price_x_zipcode = pd.DataFrame(price_x_zipcode)
#price_x_zipcode = price_x_zipcode.reset_index()
#price_x_zipcode = price_x_zipcode.rename(columns = {'price': 'Mean Price', 'zipcode': 'Zipcode'})

#fig = px.bar(price_x_zipcode, 'Zipcode', 'Mean Price')
#fig.update_xaxes(tickangle=90, nticks = 70)

### Visualizing the relationship between 'price' and 'floors'

In [737]:
#Making a new DataFrame that shows the mean selling price per number of floors
price_x_floors = kc_house_df.groupby('floors')['price'].mean()
price_x_floors = pd.DataFrame(price_x_floors)

#Renaming the columns for clarity
price_x_floors = price_x_floors.rename(columns = {'price': 'Mean Price', 'floors': 'Floors'})
price_x_floors = price_x_floors.rename_axis('Number of Floors')

#Graphing this relationship using a bar graph where x is the number of floors and y is the mean price
fig = px.bar(price_x_floors, price_x_floors.index, 'Mean Price', text = 'Mean Price', title='Mean Price Per Number of Floors',
             width=900, height = 650)

#Formating the text and color of the graph
fig.update_traces(texttemplate='%{text:.3s}', textposition='outside', marker_color='darksalmon')
fig.update_layout(font=dict(size=13))
fig.show()

### Visualizing the mean selling price per month in 2014-2015

In [765]:
#HELP
#Making a new DataFrame that shows the mean selling price per month in 2014-2015
#price_x_month_2014 = kc_house_df.loc[kc_house_df['year_sold'] == '2014']
price_x_month = kc_house_df.groupby('month_sold')['price'].mean()
price_x_month = pd.DataFrame(price_x_month)
price_x_month = price_x_month.reset_index()

#Converting the values in month_sold to int so I can order them in the DataFrame
price_x_month['month_sold'] = price_x_month['month_sold'].apply(lambda x: int(x))

#Sorting the month_sold values in descending order
price_x_month = price_x_month.sort_values(by = 'month_sold', ascending = False)

#Renaming the columns for clarity
price_x_month = price_x_month.rename(columns = {'price': 'Mean Price', 'month_sold': 'Month Sold'})

#Setting the index to 'Month Sold'
price_x_month = price_x_month.set_index('Month Sold')

#Graphing this relationship using a bar graph where x is the month and y is the mean price
fig = px.line(price_x_month, price_x_month.index, 'Mean Price', text = 'Mean Price',
                 title='Mean Selling Price Per Month in 2014-2015', width=1000, height = 700)


#Formating the text and color of the graph as well as the y-axis values
fig.update_traces(marker_color='crimson', texttemplate='%{text:.3s}', textposition='top center')
fig.update_yaxes(range=[460000, 620000])
fig.update_xaxes(range=[0, 13], tickvals=[1,2,3,4,5,6,7,8,9,10,11,12])
fig.update_layout(font=dict(size=15))
fig.show()

In [749]:
#Grouping each view level by mean price
price_x_view = kc_house_df.groupby('view')['price'].mean()

#Making a this series in a DataFrame and sorting by price
price_x_view = pd.DataFrame(price_x_view).sort_values(by = 'price')

#Renaming for clarity
price_x_view = price_x_view.rename(columns = {'price': 'Mean Price'})
price_x_view = price_x_view.rename_axis('View Rating')

#Graphing this relationship using a bar graph where x is the view rating and y is the mean price
fig = px.bar(price_x_view, price_x_view.index, 'Mean Price', text = 'Mean Price', 
             title='Mean Price Per View Rating', width=1000, height = 700)

fig.update_traces(texttemplate='%{text:.3s}', textposition='outside', marker_color='mediumaquamarine')

fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray':['EXCELLENT', 'GOOD', 'AVERAGE', 'FAIR', 'NONE']},
                  font=dict(size=13),
                  showlegend = False
                  )

### Visualizing the mean selling price per condition rating (Very Good, Good, Average, Fair/Poor)

In [738]:
#Looking at the value counts for 'condition'
condition = kc_house_df['condition']
condition.value_counts()

#Because there are signficantly less houses rated as 'Poor' and 'Fair', I will combine these two columns
price_x_condition = kc_house_df[['condition', 'price']]
price_x_condition.loc[(price_x_condition['condition'] == 'Fair')|(price_x_condition['condition'] == 'Poor'), \
                      'condition'] = 'Fair/Poor'

#Grouping each condition level by mean price
price_x_condition = price_x_condition.groupby('condition')['price'].mean()

#Making a this series in a DataFrame
price_x_condition = pd.DataFrame(price_x_condition)

#Renaming for clarity
price_x_condition = price_x_condition.rename(columns = {'price': 'Mean Price'})
price_x_condition = price_x_condition.rename_axis('Condition Rating')

#Graphing this relationship using a bar graph where x is the condition rating and y is the mean price
fig = px.bar(price_x_condition, price_x_condition.index, 'Mean Price', text = 'Mean Price', 
             title='Mean Price Per Condition Rating', width=1000, height = 650)

fig.update_traces(texttemplate='%{text:.3s}', textposition='outside', marker_color='cornflowerblue')

fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray':['Very Good', 'Good','Average','Fair/Poor']},
                  font=dict(size=13),
                  showlegend = False
                  )

### Visualizing the relationship between 'sqft_living' and 'sqft_lot'

In [764]:
#HELP
living_x_lot = kc_house_df[['sqft_living', 'sqft_lot', 'price']]
px.scatter(living_x_lot, 'sqft_living', 'sqft_lot', hover_data = ['price'], color='price', opacity = 0.7)

## Modeling

### Train Test Split

In [183]:
#Train test split with 70% of the data in the train data and 30% in the test data

#Target variable is price
y = kc_house_df['price']

#Predictor variables are every variable except price
X = kc_house_df.drop(columns=['price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [184]:
#Checking the size of the train data and the test data, respectively
print(len(X_train))
print(len(X_test))

11991
5139


### Converting 'zipcode' into categorical columns using dummy variables

In [185]:
#Using OneHotEncoder to make columns for each category in 'zipcode' for both the train and test data
#I specified drop_first = True to drop the first column as a way to avoid the dummy variable trap.
#The dropped column will become the reference category
#zipcode_cats_train = pd.get_dummies(X_train['zipcode'], drop_first = True)
#zipcode_cats_test = pd.get_dummies(X_test['zipcode'], drop_first = True)

#Concating X_train and zipcode_cats to make one large dataframe and reassigning it to X_train
#X_train = pd.concat([X_train, zipcode_cats_train], axis = 1)

#Concating X_train and zipcode_cats to make one large dataframe and reassigning it to X_train
#X_test = pd.concat([X_test, zipcode_cats_test], axis = 1)

In [186]:
#Because we made 'zipcode' into multiple categorical columns using pd.get_dummies, we can drop 'zipcode'
#We do this to both the train data and the test data

#Dropping 'zipcode' from the train data
#X_train.drop(columns = ['zipcode'], inplace = True)

#Dropping 'zipcode' from the test data
#X_test.drop(columns = ['zipcode'], inplace = True)

### Making the baseline model

In [ ]:
#The baseline model will include all the numerical columns
base = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']

X_train[base]
X_test[base]

In [ ]:
lr = LinearRegression()

In [ ]:
#Fitting the train data
lr.fit(X_train[base], y_train)

In [ ]:
#Checking the R-squared of our baseline model
lr.score(X_train[base], y_train)

In [ ]:
#Validating my model
lr.score(X_test[base], y_test)

## Regression Results

## Conclusions